In [1]:
# Question 1

# well there are 16 number of replications
# because 8 * 2

In [ ]:
# Question 2

# Each chip is gathering at once 3/4 of the size of the array for
# allgather_X

X = 4
Y = 4
Z = 4
B = 1024
D = 4096
total_xy_bytes = 2 * 1024 * 4096
bidirectional_ici_bandwidth = 9e10 # one way

allgather_x_time = (total_xy_bytes / Y) / bidirectional_ici_bandwidth # Y remains sharded

allgather_xy_time = total_xy_bytes / (bidirectional_ici_bandwidth * 2)

allreduce_time = 2 * total_xy_bytes / (bidirectional_ici_bandwidth * X * Y)

print(f"allgather_x_time: {allgather_x_time}")
print(f"allgather_xy_time: {allgather_xy_time}")
print(f"allreduce_time: {allreduce_time}")

# Idea to solve is that we just think about what the final array size should look like
# that then gives us the total size of the transfer which is our numerator.
# Our denominator is the number of axes in which we can transfer the data (X, Y, Z) and/or
# if they are bidirectional links.

allgather_x_time: 2.330168888888889e-05
allgather_xy_time: 4.660337777777778e-05
allreduce_time: 1.1650844444444444e-05


In [ ]:
# Question 3

b_bytes = 128 * 2
X = 4
total_time = b_bytes / (bidirectional_ici_bandwidth)
total_time # but actually latency bounded so this is just 2 microseconds

2.8444444444444444e-09

In [ ]:
# Question 4

# Strategy one:
# T_comm = DF / W_ici
# T_math = BDF / FLOPs

# Strategy two:
# T_comm = 2 * BF / W_ici
# T_math = BD_xF / FLOPs

# Strategy two is clearly better in the case where B << D and we end up doing the 
# computation on the sharded array which is cheaper.

In [ ]:
# Question 5

#  A[I_xyz, J] * B[J, K] -> C[I, K]
# all gather on XYZ plane 
# IJ / (W_ici * 3)

# if we allgather first which we shouldn't...
#  T_comm = IJ / (W_ici * 3)
#  T_math = IJK / FLOPs

# allgather at the end
# allgather(C[I_{xyz}, K])
# T_math = IJK / (FLOPs * XYZ)
# T_comm = IK / (W_ici * 3)

# A[I, J_xyz] * B[J, K]
# all gather J on XYZ plane... same communication cost
# T_comm = IJ / (W_ici * 3)
# T_math = IJK / FLOPs

# A[I, J] * B[J_xyz, K]
# all gather J on XYZ plane .. contraction on the second array could be cheaper if K << I?
# T_comm = JK / (W_ici * 3)
# T_math = IJK / FLOPs

# A[I, J_xyz] * B[J_xyz, K] -> C[I, K] {U_xyz}
# Local computation then all reduce
# T_math =  IJ / (FLOPs * XYZ)
# T_comm = 2 * IK / (W_ici * 3)  # 2x because of all reduce which can be expensive

# A[I, J] * B[J, K] -> C[I, K] fully replicated
# T_comm = 0
# T_math = IJK / FLOPs

In [ ]:
# Question 6

# A[I_x, J_y] * B[J_y, K] -> C[I_x, K] {U_y}
# All reduce needed at the end
# T_math = I_x * J_y * K / FLOPs
# T_comm = 2 * (I_x * K) / (W_ici)

# A[I_x, J] * B[J_x, K_y] -> C[I_x, K_y]
# 1. all gather along X for weight B -> A[I_x, J] * B[J, K_y]
# T_math = I_x * J * K_y / FLOPs
# T_comm = J * K_y / W_ici

# A[I_x, J] * B[J, K_y] -> C[I_x, K_y]
# T_math = I_x * J * K_y / FLOPs
# T_comm = 0

In [ ]:
# Question 7

# B[D, F], C[F, D]
# whole block is C * B * x with x[B, D]
# D = 8192, F = 32768, B = 128
# data type: bf16
# 